# PSI On PPU

PSI([Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection))是一种使用密码学方法，获取两份数据内容的交集的算法。PSI过程中不泄露任务交集以外的信息。隐私求交技术可解决需要进行数据合作的双方需要获取、分析数据的交集，同时不希望泄露无关内容的问题，通常在联合营销等场景中被应用。

PPU实现了以下两种协议：
- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：半诚实协议，基于公钥加密，适合小规模数据集
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：半诚实协议，基于cuckoo hashing和OT，适合大规模数据集

在开始之前，我们需要对环境进行初始化。以下在单机创建了alice, bob, carol三个节点以模拟多个参与方。

In [1]:
import secretflow as sf

sf.init(['alice', 'bob', 'carol'], num_cpus=8, log_to_driver=False)

## 准备数据集

首先，我们需要一份数据集用于构建两方、三方垂直切分的场景。为了简单起见，这里使用[iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)数据集，我们为其添加两列用于后续单列和多列求交的演示
- uid：样本唯一ID
- month：模拟样本按月生成的场景，前50%样本是1月生成，后50%样本是2月生成

In [2]:
import numpy as np
from sklearn.datasets import load_iris

data, target = load_iris(return_X_y=True, as_frame=True)
data['uid'] = np.arange(len(data)).astype('str')
data['month'] = ['Jan'] * 75 + ['Feb'] * 75

data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),uid,month
0,5.1,3.5,1.4,0.2,0,Jan
1,4.9,3.0,1.4,0.2,1,Jan
2,4.7,3.2,1.3,0.2,2,Jan
3,4.6,3.1,1.5,0.2,3,Jan
4,5.0,3.6,1.4,0.2,4,Jan
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,145,Feb
146,6.3,2.5,5.0,1.9,146,Feb
147,6.5,3.0,5.2,2.0,147,Feb
148,6.2,3.4,5.4,2.3,148,Feb


在实际场景中，样本数据是由各个参与方提供的，并且需要提前约定好用于求交的字段：
- 求交字段可以是单个或多个，但是必须是各方共同拥有
- 求交字段必须唯一，如有重复，需要提前去重

例如，以下是由alice提供的用于PSI求交的数据，求交字段为`uid`和`month`，可以看到[1, 'Jan']有重复，在PSI求交前需要去重。
```
alice.csv
---------
uid   month   c0
1     Jan     5.8
2     Jan     5.4
1     Jan     5.8
1     Feb     7.4
```
去重后的数据为
```
alice.csv
---------
uid   month   c0
1     Jan     5.8
2     Jan     5.4
1     Feb     7.4
```

我们对iris数据进行三次随机采样以模拟alice, bob, carol提供的数据，此时三份数据处于未对齐状态。

In [3]:
import os

os.makedirs('.data', exist_ok=True)
da, db, dc = data.sample(frac=0.9), data.sample(frac=0.8), data.sample(frac=0.7)

da.to_csv('.data/alice.csv', index=False)
db.to_csv('.data/bob.csv', index=False)
dc.to_csv('.data/carol.csv', index=False)

## 两方PSI

我们在物理设备上虚拟出三个逻辑设备
- alice, bob：PYU设备，负责参与方本地的明文计算
- ppu：PPU设备，由alice和bob构成，负责两方的密文计算

In [4]:
alice, bob = sf.PYU('alice'), sf.PYU('bob')
ppu = sf.PPU(sf.utils.testing.cluster_def(['alice', 'bob']))

### 单列求交

接下来，我们使用`uid`对两方数据进行求交，PPU提供了`psi_csv`以csv文件作为输入，生成求交后的csv文件，默认使用KKRT协议。

In [5]:
input_path = {alice: '.data/alice.csv', bob: '.data/bob.csv'}
output_path = {alice: '.data/alice_psi.csv', bob: '.data/bob_psi.csv'}
ppu.psi_csv('uid', input_path, output_path)

为了校验结果的正确性，我们使用[pandas.DataFrame.join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)对da, db进行inner join。可以看到，两方数据已经按照`uid`进行了对齐，并依据其字典顺序进行了排序。

In [6]:
import pandas as pd

df = da.join(db.set_index('uid'), on='uid', how='inner', rsuffix='_bob', sort=True)
expected = df[da.columns].astype({'uid': 'int64'}).reset_index(drop=True)

da_psi = pd.read_csv('.data/alice_psi.csv')
db_psi = pd.read_csv('.data/bob_psi.csv')

pd.testing.assert_frame_equal(da_psi, expected)
pd.testing.assert_frame_equal(db_psi, expected)

da_psi

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),uid,month
0,6.3,3.3,6.0,2.5,100,Feb
1,5.8,2.7,5.1,1.9,101,Feb
2,7.1,3.0,5.9,2.1,102,Feb
3,6.3,2.9,5.6,1.8,103,Feb
4,6.5,3.0,5.8,2.2,104,Feb
...,...,...,...,...,...,...
101,5.6,2.7,4.2,1.3,94,Feb
102,5.7,2.9,4.2,1.3,96,Feb
103,6.2,2.9,4.3,1.3,97,Feb
104,5.1,2.5,3.0,1.1,98,Feb


### 多列求交

我们也可以使用多个字段进行求交，以下演示了使用`uid`和`month`对两方数据进行求交。在实现上，多个字段被拼接成一个字符串，因此请保证多列复合主键(Composite Primary Key)没有重复。

In [7]:
ppu.psi_csv(['uid', 'month'], input_path, output_path)

同样的，我们使用pandas.DataFrame.join验证结果正确性，可以看到，两方数据已经按照`uid`和`month`进行了对齐，并依据其字典顺序进行了排序。

In [8]:
df = da.join(db.set_index(['uid', 'month']), on=['uid', 'month'], how='inner', rsuffix='_bob', sort=True)
expected = df[da.columns].astype({'uid': 'int64'}).reset_index(drop=True)

da_psi = pd.read_csv('.data/alice_psi.csv')
db_psi = pd.read_csv('.data/bob_psi.csv')

pd.testing.assert_frame_equal(da_psi, expected)
pd.testing.assert_frame_equal(db_psi, expected)

## 三方PSI

首先，我们增加一个第三方`carol`，并为其创建一个PYU设备，同时创建一个由三方构建的PPU设备。

In [9]:
carol = sf.PYU('carol')
ppu_3pc = sf.PPU(sf.utils.testing.cluster_def(['alice', 'bob', 'carol']))

然后，以`uid`和`month`为复合主键进行三方求交，需要注意的是，三方求交暂时只支持ECDH协议。

In [10]:
input_path = {alice: '.data/alice.csv', bob: '.data/bob.csv', carol: '.data/carol.csv'}
output_path = {alice: '.data/alice_psi.csv', bob: '.data/bob_psi.csv', carol: '.data/carol_psi.csv'}
ppu_3pc.psi_csv(['uid', 'month'], input_path, output_path, protocol='ecdh')

同样的，我们使用pandas.DataFrame.join验证结果正确性。

In [11]:
keys = ['uid', 'month']
df = da.join(db.set_index(keys), on=keys, how='inner', rsuffix='_bob', sort=True).join(
    dc.set_index(keys), on=keys, how='inner', rsuffix='_carol', sort=True)
expected = df[da.columns].astype({'uid': 'int64'}).reset_index(drop=True)

da_psi = pd.read_csv('.data/alice_psi.csv')
db_psi = pd.read_csv('.data/bob_psi.csv')
dc_psi = pd.read_csv('.data/carol_psi.csv')

pd.testing.assert_frame_equal(da_psi, expected)
pd.testing.assert_frame_equal(db_psi, expected)
pd.testing.assert_frame_equal(dc_psi, expected)

## What's Next

OK！通过本教程，我们已经了解了如何通过PPU进行两方和三方数据求交。完成数据求交后，我们可以在对齐后的数据集上进行机器学习建模
- [Logistic Regression On PPU](./LR_On_PPU.ipynb)：使用JAX在PPU上进行逻辑回归建模
- Neural Network on PPU：使用JAX在PPU进行神经网络建模
- Basic Split Learning：使用TensorFlow和拆分学习进行神经网络建模